In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').getOrCreate()

In [11]:
spark_df = (spark.read.format("csv").options(header="true").load("./data/spotify_artists.csv"))

## Profiling the Data:

In [ ]:
# Show a description (summary) of the Spark DataFrame.
spark_df.describe

In [ ]:
# Print the schema of the DataFrame.
spark_df.printSchema()

In [ ]:
# Select and show just the first 10 values in the 'name' and 'genres' columns.
spark_df.select(spark_df.name, spark_df.genres).show(10)

## Cleaning the Data:

In [3]:
# Where the genre is an empty list, replace it with ['elevator music']
from pyspark.sql.functions import regexp_replace
spark_df.where(spark_df.genres == "[]").show(5)
spark_df = spark_df.withColumn('genres', regexp_replace('genres', r"\[\]", "['elevator music']"))
spark_df.where(spark_df.genres == "['elevator music']").show(5)


+-----+-----------------+---------+------+--------------------+--------------+--------------------+---------------+------+
|index|artist_popularity|followers|genres|                  id|          name|            track_id|track_name_prev|  type|
+-----+-----------------+---------+------+--------------------+--------------+--------------------+---------------+------+
|    1|               22|      313|    []|1dLnVku4VQUOLswwD...|The Grenadines|4wqwj0gA8qPZKLl5W...|       track_30|artist|
|    5|               43|       81|    []|38VBjthd0szbS6wpD...|        Filhos|453KeZU566kjNfs1I...|       track_15|artist|
|    6|               34|     8358|    []|36mHwYa65L0WZbAXY...|          Eloq|6hC5Tl0S5aQCw646J...|       track_43|artist|
|    7|                7|      158|    []|1jJyy00XfxjB4tMAv...|        Fravær|5005eeCJ9KTFAAzh2...|       track_48|artist|
|    8|               21|       30|    []|10A8WbBJ0zW8MnvTs...| Camille Pépin|56fBsCXwpBnxe4JkF...|       track_73|artist|
+-----+---------

In [4]:
# For the columns 'artist_popularity' and 'followers', cast the data type as integers.
from pyspark.sql.types import IntegerType

spark_df = spark_df.withColumn('artist_popularity', spark_df['artist_popularity'].cast(IntegerType()))
spark_df = spark_df.withColumn('followers', spark_df['followers'].cast(IntegerType()))
spark_df.select('artist_popularity', 'followers').printSchema()

root
 |-- artist_popularity: integer (nullable = true)
 |-- followers: integer (nullable = true)



In [5]:
# Sort the data in descending order by number of followers.

spark_df = spark_df.orderBy('followers', ascending=False)
spark_df.show(10)

+-----+-----------------+---------+--------------------+--------------------+-------------+--------------------+---------------+------+
|index|artist_popularity|followers|              genres|                  id|         name|            track_id|track_name_prev|  type|
+-----+-----------------+---------+--------------------+--------------------+-------------+--------------------+---------------+------+
|55251|               92| 41561693|   ['pop', 'uk pop']|6eUKZXaKkcviH0Ku9...|   Ed Sheeran|7qiZfU4dY1lWllzX7...|       track_35|artist|
|53392|               98| 34680740|['canadian hip ho...|3TVXtAsR1Inumwj47...|        Drake|116H0KvKr2Zl4RPuV...|       track_71|artist|
|52620|               90| 30560149|['dance pop', 'po...|5pKCCKE2ajJHZ9KAi...|      Rihanna|2Ce5IyMlVRVvN997Z...|       track_38|artist|
|54447|               88| 26824224|['canadian pop', ...|1uNFoZAHBGtllmzzn...|Justin Bieber|3A7qX2QjDlPnazUsR...|        track_2|artist|
|42872|              100| 26309771|['dance pop',

In [6]:
# 'artist_popularity' is a rank out of 100. Write a user defined function that will divide each popularity value by 100. Rename the column 'popularity_percent'.
from pyspark.sql.functions import udf

percent = udf(lambda x: x/100)

spark_df = spark_df.withColumn('popularity_percent', percent('artist_popularity')).show(10)

+-----+-----------------+---------+--------------------+--------------------+-------------+--------------------+---------------+------+------------------+
|index|artist_popularity|followers|              genres|                  id|         name|            track_id|track_name_prev|  type|popularity_percent|
+-----+-----------------+---------+--------------------+--------------------+-------------+--------------------+---------------+------+------------------+
|55251|               92| 41561693|   ['pop', 'uk pop']|6eUKZXaKkcviH0Ku9...|   Ed Sheeran|7qiZfU4dY1lWllzX7...|       track_35|artist|              0.92|
|53392|               98| 34680740|['canadian hip ho...|3TVXtAsR1Inumwj47...|        Drake|116H0KvKr2Zl4RPuV...|       track_71|artist|              0.98|
|52620|               90| 30560149|['dance pop', 'po...|5pKCCKE2ajJHZ9KAi...|      Rihanna|2Ce5IyMlVRVvN997Z...|       track_38|artist|               0.9|
|54447|               88| 26824224|['canadian pop', ...|1uNFoZAHBGtllm

## Extracting Information

In [12]:
# Show only the values in the DataFrame that have 'Queen' in the name

spark_df.select('*').filter(spark_df.name.contains('Queen')).show(5)

# spark_df.createOrReplaceTempView('spotify')
# spark.sql("SELECT * FROM spotify WHERE name LIKE '%Queen%'").show(5)


+-----+-----------------+---------+--------------------+--------------------+--------------------+--------------------+---------------+------+
|index|artist_popularity|followers|              genres|                  id|                name|            track_id|track_name_prev|  type|
+-----+-----------------+---------+--------------------+--------------------+--------------------+--------------------+---------------+------+
|   40|               16|      695|                  []|4SK9OzAA0K00NVsXA...|       Queen Machine|6u3RWvO7ZIIdVci1N...|       track_56|artist|
|  901|               43|    18224|           ['strut']|71WL5bNm5jPPpwpDc...|  Bob the Drag Queen|5IsdA6g8IFKGmC1xl...|        track_8|artist|
| 1518|               30|     2297|           ['benga']|2FzYw9fn2ZtQ7sZma...|Muthoni Drummer Q...|4F0e4hx3bASeaqLqS...|       track_45|artist|
| 2152|               22|     3244|['afropop', 'kwai...|5LFWp4p0pMURif2d7...|Mahlathini & The ...|6WbcheHRcJNMaDIkO...|       track_15|artist|

In [ ]:
# Group the data by artist popularity, and show the count for each group.

spark_df.groupBy('artist_popularity').sum('artist_popularity').show(10)

In [ ]:
# save the DataFrame as a Parquet file in the /data directory.

spark_df.write.parquet("./data/spotify_artists.parquet")